In [4]:
import sqlite3
import numpy as np
import datetime

import trueskill
from collections import defaultdict

In [2]:
con = sqlite3.connect('soccer.sqlite')
c = con.cursor()
data = [x for x in c.execute('select FTR, homeID, awayID, date from match  WHERE ligaID IS 19 ORDER BY date')]
con.close()

In [5]:
env = trueskill.TrueSkill(draw_probability=0.25)

teams = defaultdict(env.create_rating)

date_old = datetime.date(*[int(x) for x in data[0][3].split('-')])

for m in data:
    t1 = int(m[1]/1000000)
    t2 = int(m[2]/1000000)
    
    r1 = teams[t1]
    r2 = teams[t2]
    
    date_new = datetime.date(*[int(x) for x in m[3].split('-')])
    
    if datetime.timedelta(53) < abs(date_new-date_old):
        r1 = env.create_rating(r1.mu, 3*r1.sigma)
        r2 = env.create_rating(r2.mu, 3*r2.sigma)
        
    date_old = date_new
    if m[0] == 'H':
        r1,r2 = env.rate_1vs1(r1,r2)
    elif m[0] == 'D':
        r1,r2 = env.rate_1vs1(r1,r2,drawn=True)
    elif m[0] =='A':
        r2,r1 = env.rate_1vs1(r2,r1)
    else:
        print('wrong result: '+m[0])
    
    teams[t1] = r1
    teams[t2] = r2
      
    

AttributeError: 'int' object has no attribute 'split'

In [83]:
con = sqlite3.connect('soccer.sqlite')
c = con.cursor()
team_names = {x[0]:x[1] for x in c.execute('SELECT sofifaTeamID, name FROM team')}
con.close()

In [104]:
ranks = []
for key in teams.keys():
    ranks.append([team_names[key], str(teams[key].mu)[:5], str(teams[key].sigma)[:4]])

ranks = np.array(ranks)
ranks[np.argsort(ranks[:,1] )][::-1][-15:]

array([['Morecambe', '21.19', '0.75'],
       ['Balikesirspor', '21.14', '1.42'],
       ['SAI Kayseri Erciyesspor', '21.12', '1.37'],
       ['Livorno', '21.00', '0.96'],
       ['Dagenham and Redbridge', '20.92', '0.76'],
       ['Hartlepool United', '20.89', '0.75'],
       ['Rot-Weiss Ahlen', '20.61', '1.42'],
       ['FC Dordrecht', '20.56', '1.32'],
       ['Cordoba CF', '20.36', '1.21'],
       [' SSV Jahn Regensburg', '20.23', '1.19'],
       ['Pescara', '20.02', '1.27'],
       ['Crotone', '19.29', '2.85'],
       ['Stockport County', '19.03', '1.23'],
       ['Macclesfield Town', '18.40', '1.59'],
       ['Benevento Calcio', '12.40', '3.86']], 
      dtype='<U25')

In [105]:
con = sqlite3.connect('soccer.sqlite')
c = con.cursor()
test_date = [x for x in c.execute('SELECT * FROM patch')]
con.close()

In [117]:
datetime.timedelta(53) < abs(datetime.date(*[int(x) for x in test_date[1][3].split('-')])-datetime.date(*[int(x) for x in test_date[2][3].split('-')]))

False

In [7]:
data[0][3]

2009

In [118]:
abs(datetime.timedelta(-3))

datetime.timedelta(3)